In [1]:
from gensim.models import Word2Vec,KeyedVectors
import pickle
import os
import numpy as np
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve,roc_auc_score,recall_score,f1_score,precision_score,confusion_matrix

labels = np.zeros(25000)
labels[:12500] = 1

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# EXTRACT FEATURES

In [0]:
with open('/content/drive/My Drive/all data/tr','rb') as f:
  tr = pickle.load(f)
with open('/content/drive/My Drive/all data/te','rb') as f:
  te = pickle.load(f)
tr.extend(te)
len(tr)

50000

In [0]:
model = Word2Vec(tr,size=100,
        window=10,
        min_count=0,
        workers=10,
        iter=100)
        

Word2Vec.save(model,'/content/drive/My Drive/WORD2VEC/wv_model')

In [0]:
#model = Word2Vec.load('/content/drive/My Drive/WORD2VEC/wv_model')
#model.train(tr,total_examples=50000,
#        epochs=10)

#Word2Vec.save(model,'/content/drive/My Drive/WORD2VEC/wv_model')


a = KeyedVectors.load('/content/drive/My Drive/WORD2VEC/wv_model',mmap = 'r')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
w2v_feats = []
for doc in tr:
  w2v_feats.append(np.mean(a.wv.__getitem__(doc),axis = 0))
#with open('/content/drive/My Drive/WORD2VEC/w2v_feats','wb') as f:
#  pickle.dump(w2v_feats,f)

# LOAD FEATURES

In [0]:
with open('/content/drive/My Drive/WORD2VEC/w2v_feats','rb') as f:
  feats = pickle.load(f)
with open('/content/drive/My Drive/WORD2VEC/w2v_feats_sw','rb') as f:
  feats_sw = pickle.load(f)

## **Nive Bayes**

In [0]:
gnb = BernoulliNB()
gnb_model = gnb.fit(feats[:25000],labels)
#with open('/content/drive/My Drive/nb/w2v_sw','wb') as f:
#  pickle.dump(gnb_model_sw,f)

In [0]:
y_pred = gnb_model.predict(feats[25000:])
#with open('/content/drive/My Drive/nb/y_pred_W2V','wb') as f:
#  pickle.dump(y_pred,f)


In [0]:
f1_score(y_pred,labels)

0.7222634049913517

In [0]:
gnb = BernoulliNB()
gnb_model_sw = gnb.fit(feats_sw[:25000],labels)

In [0]:
y_pred_sw = gnb_model_sw.predict(feats_sw[:25000])
#with open('/content/drive/My Drive/nb/y_pred_W2V_sw','wb') as f:
#  pickle.dump(y_pred_sw,f)
f1_score(y_pred_sw,labels)

0.6264481610987066

In [0]:
with open('/content/drive/My Drive/nb/y_pred_W2V_sw','rb') as f:
  y_pred_sw = pickle.load(f)
with open('/content/drive/My Drive/nb/y_pred_W2V','rb') as f:
  y_pred = pickle.load(f)

In [0]:
print(np.mean(y_pred == labels))
print(np.mean(y_pred_sw == labels))

0.74812
0.66768


# Random Forest

In [0]:
rf = RandomForestClassifier()
w2v = rf.fit(feats[:25000],labels)
y_pred_w2v = w2v.predict(feats[25000:])

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [0]:
rf = RandomForestClassifier()
w2v_sw = rf.fit(feats_sw[:25000],labels)
y_pred_w2v_sw = w2v_sw.predict(feats_sw[25000:])

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [0]:
with open('/content/drive/My Drive/rf/y_pred_W2V','rb') as f:
  y_pred_w2v = pickle.load(f)
with open('/content/drive/My Drive/rf/y_pred_W2V_sw','rb') as f:
  y_pred_w2v_sw = pickle.load(f)

In [0]:
print(np.mean(y_pred_w2v == labels))
print(np.mean(y_pred_w2v_sw == labels))

0.76136
0.70568


# Decision Tree

In [0]:
dt = DecisionTreeClassifier()
w2v_sw = dt.fit(feats_sw[:25000],labels)
y_pred_w2v_sw = w2v_sw.predict(feats_sw[25000:])

In [0]:
dt = DecisionTreeClassifier()
w2v = dt.fit(feats[:25000],labels)
y_pred_w2v = w2v.predict(feats[25000:])

In [0]:
with open('/content/drive/My Drive/dt/y_pred_W2V','rb') as f:
  y_pred_w2v = pickle.load(f)
with open('/content/drive/My Drive/dt/y_pred_W2V_sw','rb') as f:
  y_pred_w2v_sw = pickle.load(f)

In [0]:
print(np.mean(y_pred_w2v == labels))
print(np.mean(y_pred_w2v_sw == labels))

0.7006
0.6414


# SVM

In [0]:
s = svm.LinearSVC(max_iter=500)
s.fit(feats[:25000],labels)
y_pred = s.predict(feats[25000:])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
s_sw = svm.LinearSVC(max_iter=500)
s_sw.fit(feats_sw[:25000],labels)
y_pred_sw = s_sw.predict(feats_sw[25000:])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
with open('/content/drive/My Drive/svm/y_pred_W2V','rb') as f:
  y_pred = pickle.load(f)
with open('/content/drive/My Drive/svm/y_pred_W2V_sw','rb') as f:
  y_pred_sw = pickle.load(f)

In [0]:
print(np.mean(y_pred == labels))
print(np.mean(y_pred_sw == labels))

0.86364
0.84784
